# In-Class Assignment 20: Neutron Star Equations of State

**In-Class Only, Not Submitted for Credit**, Credit: adapted from Michael Zingale.

## Learning Objectives

- Develop a qualitative understanding of the M-R relationship for neutron stars
- Identify the impact of the compressibility (squishiness) on the M-R relation and the maximum NS mass
- Identify limits to current NS equation of state modeling and measurements

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Numerical Integrator and 2 EoS (provided)

For this section just read the text and run the cells. You can come back to the code on your own time if you are interested!

Here's a general integration class that implemented 4th order Runge-Kutta.  It takes
a `rhs` function of the form:

```
rhs(r, Y, rho_func)
```

that returns the derivatives $d{\bf Y}/dr$ for each of the variables in the array ${\bf Y}$.
Here, `rho_func` is a function `rho_func(pres)` that takes the pressure and returns the
density according to our EOS.

In [ ]:
class Integrator:
    """ this is a simple RK4 integrator with a uniform step size
    that can accept a list of stopping criteria to halt the integration.

    There is one list element for each variable.  An entry of "None"
    means that we have no stopping criteria for that variable.  A tuple
    of the form ("LT", val) or ("GT", val), means that we stop if
    y[n] < val (or y[n] > val)
    """

    def __init__(self, rhs, *, stop_cond=None, rhs_args=None):
        self.rhs = rhs
        self.stop_cond = stop_cond
        if rhs_args is None:
            rhs_args = []
        self.args = rhs_args

    def rk4_step(self, x, y, dx):

        dydx1 = self.rhs(x, y, *self.args)
        y1 = y + 0.5*dx*dydx1
        if self.check_stop_cond(y1) < 0: return None

        dydx2 = self.rhs(x + 0.5*dx, y1, *self.args)
        y2 = y + 0.5*dx*dydx2
        if self.check_stop_cond(y2) < 0: return None

        dydx3 = self.rhs(x + 0.5*dx, y2, *self.args)
        y3 = y + dx*dydx3
        if self.check_stop_cond(y3) < 0: return None

        dydx4 = self.rhs(x + dx, y3, *self.args)

        y_final = y + (dx/6.0)*(dydx1 + 2.0*dydx2 + 2.0*dydx3 + dydx4)
        if self.check_stop_cond(y_final) < 0: return None

        return y_final

    def check_stop_cond(self, y):
        # check our stopping conditions
        if not self.stop_cond is None:
            for n, cond in enumerate(self.stop_cond):
                if cond is None: continue
                c, v = cond
                if c == "LT":
                    if y[n] < v: return -1
                elif c == "GT":
                    if ynew[n] > v: return -1
                else:
                    sys.exit("invalid condition")

        return 0

    def integrate(self, initial_conditions, dx, xmax, tol=1.e-8):
        ic = np.array(initial_conditions)
        N = len(ic)
        sol = {}
        for n in range(N):
            sol[f"y{n}"] = [ic[n]]

        x = 0.0
        sol["x"] = [x]

        y = initial_conditions
        
        while (x < xmax and dx > tol * xmax):
            # advance for a step
            ynew = self.rk4_step(x, y, dx)

            if ynew is None:
                # we hit our stopping condition
                # cut the stepsize so we can try to get closer
                dx /= 2
                continue

            # store the solution
            x += dx
            sol["x"].append(x)
            for n in range(N):
                sol[f"y{n}"].append(ynew[n])

            y = ynew

        return sol

Some physical constants (CGS)

In [ ]:
G = 6.67e-8
c = 3.e10
h = 6.63e-27
m_u = 1.66e-24

Here's the righthand side function for the TOV equation

In [ ]:
def rhs_tov(r, y, rho_func):
    m = y[0]
    P = y[1]

    rho = rho_func(P)
    
    dmdr = 4.0 * np.pi * r**2 * rho
        
    if r == 0.0:
        dPdr = 0.0
    else:
        rho_term = rho + P / c**2
        m_term = m + 4.0 * np.pi * r**3 * P / c**2
        metric_term = (1.0 - 2.0 * G * m / (r * c**2))
        dPdr = -(G / r**2) * rho_term * m_term / metric_term

    return np.array([dmdr, dPdr])

## Degenerate neutrons

For zero-T neutron degeneracy, the number density of neutrons is just the integral of the distribution function, $n(p)$, over all momenta:

$$n = \int n(p) d^3 p = 4\pi \int_0^\infty n(p) p^2 dp$$

but at zero temperature the distribution function is just a step function:

$$n(p) = \frac{2}{h^3}\left \{ \begin{array}{cc} 1 & \mbox{if } p < p_F \\
                                    0 & \mbox{otherwise} \end{array}\right .$$
                                    
This gives:

$$n = \frac{\rho}{m_u} = \frac{8\pi}{3h^3} p_F^3 = \frac{8\pi}{3} \left (\frac{m_u c}{h} \right )^3 x_F^3$$

where $x_F = p_F / (m_u c)$.

We can then compute the pressure as:

$$P = \frac{1}{3} \int n(p) v p d^3 p$$

where we now need to use the general expression for velocity, 

$$v = \frac{p}{m_u} \left [ 1 + \left ( \frac{p}{m_u c} \right)^2 \right ]^{-1/2}$$

This gives:

$$P = \frac{\pi}{3} \left ( \frac{m_u c}{h} \right )^3 m_u c^2 f(x)$$

with

$$f(x) = x (2x^2 - 3) (1 + x^2)^{1/2} + 3 \sinh^{-1}(x)$$

In [ ]:
from scipy.optimize import brentq

def pres_deg(rho):
    # compute the Fermi momentum x = p / mc
    B = 8 * np.pi * m_u / 3.0 * (m_u * c / h)**3
    x = np.cbrt(rho / B)
    
    f = x * (2 * x**2 - 3) * np.sqrt(1 + x**2) + 3 * np.arcsinh(x)
    A = (np.pi / 3) * (m_u * c / h)**3 * m_u * c**2
    
    return A * f
    
def rho_deg(pres):
    
    A = (np.pi / 3) * (m_u * c / h)**3 * m_u * c**2
    f = pres / A
    
    # find the x that gives us this f
    x = brentq(lambda x: f - (x * (2 * x**2 - 3) * np.sqrt(1 + x**2) + 3 * np.arcsinh(x)), 0, 10)
    
    B = 8 * np.pi * m_u / 3.0 * (m_u * c / h)**3
    return B * x**3 

## A polytropic nuclear EOS

Polytropic equations of state are often used as an approximation in computational models. 

A simple polytopic approximation to a nuclear equation of state (the SLy4 EOS) is:


$$P = K \left ( \frac{\rho}{\rho_0} \right )^\gamma$$

If we choose $K = 3.5 \mathrm{MeV~fm^{-3}}$ and $\rho_0 = 150 \mathrm{MeV~fm^{-3}}$, we approximate the behavior of the SLy4 nuclear EOS (thanks to Tianqi Zhao for these approximations) with $\gamma = 2.5$.

In CGS units, using $\gamma = 2.5$, we can take $K = 4.80\times 10^{-3}~\mathrm{erg~cm^{-3}}$ and write this as:

$$P = K \rho^\gamma$$

In [ ]:
def pres_nuc(rho):
    gamma = 2.5
    K = 4.8e-3
    
    return K * rho**gamma

def rho_nuc(pres):
    gamma = 2.5
    K = 4.8e-3
    
    return (pres / K)**(1.0 / gamma)

## Activity Begins Here

## a. - Case I: Neutron Degeneracy Pressure

1. Evaluate the first cell below, which loops over central density, integrates the neutron star structure according to the TOV eqn and EoS, and stores the mass and radius corresponding to each central density.
2. Plot the resulting M vs. R in the second cell below, and mark the maximum NS mass.

In [ ]:
ns1 = Integrator(rhs_tov, rhs_args=[rho_deg],
                 stop_cond=[("LT", 0.0), ("LT", 0.0)])

R_max = 5.e6
N = 1000

# my vector is [m, p]
mass_ns1 = []
radius_ns1 = []
for rhoc in np.logspace(np.log10(3.e14), np.log10(5.e16), 100):

    Pc = pres_deg(rhoc)

    sol = ns1.integrate([0.0, Pc], R_max/N, R_max)

    mass_ns1.append(sol["y0"][-1]/2.e33)
    radius_ns1.append(sol["x"][-1]/1.e5)

In [ ]:
# a results here

#plot M over R:
plt.plot()

#get maximum mass:
#np.max(array) will return the maximum value in the array
max_mass_ns1 = 

#get radius at maximum mass:
# np.argmax(array) returns the index of that array's maxiumum value
# so array2[np.argmax(array1)] will give the element of array2 corresponding to the max of array1
radius_maxmass_ns1 = 

#plot marker:
plt.plot(radius_maxmass_ns1,max_mass_ns1,"o",color='green')

#labels:
plt.xlabel('Radius (km)')
plt.ylabel(r'Mass ($M_{\odot}$)')
plt.title('M vs R for Neutron Degeneracy')

With your group, discuss:
> 3. The maximum mass and shape of the plot. What do they mean? Are there any features you find confusing or concerning?
> 4. Is neutron degeneracy pressure sufficient to explain the NS we observe? If not, what other forces could be contributing to the pressure in NS cores?


## b - Case II: A Nuclear EOS

Next, let's use a more realistic equation of state, using the analytical form for the nuclear EoS defined above. 


1. Evaluate the first cell below, which is the same as the previous one except using the nuclear EoS.
2. Plot the resulting M vs R in the second cell below, and mark the maximum NS mass.

In [ ]:
ns2 = Integrator(rhs_tov, rhs_args=[rho_nuc],
                 stop_cond=[("LT", 0.0), ("LT", 0.0)])

R_max = 5.e6
N = 1000

# my vector is [m, p]
mass_ns2 = []
radius_ns2 = []
for rhoc in np.logspace(np.log10(3.e14), np.log10(5.e16), 100):

    Pc = pres_nuc(rhoc)

    sol = ns2.integrate([0.0, Pc], R_max/N, R_max)

    mass_ns2.append(sol["y0"][-1]/2.e33)
    radius_ns2.append(sol["x"][-1]/1.e5)

In [ ]:
# b results here

#plot M over R:
plt.plot()

#get maximum mass:
#np.max(array) will return the maximum value in the array
max_mass_ns2 = 

#get radius at maximum mass:
# np.argmax(array) returns the index of that array's maxiumum value
# so array2[np.argmax(array1)] will give the element of array2 corresponding to the max of array1
radius_maxmass_ns2 = 

#plot marker:
plt.plot(radius_maxmass_ns2,max_mass_ns2,"o",color='green')

#labels:
plt.xlabel('Radius (km)')
plt.ylabel(r'Mass ($M_{\odot}$)')
plt.title('M vs R for Nuclear EoS')

With your group, discuss:
> 3. The maximum mass and shape of the plot. What do they mean? Are there any features you find confusing or concerning?
> 4. Look at the M vs. R plot on the slides, which has an EoS plot including a SLy model (like the one we are approximating here). Compare the shape of your SLy curve to the one in the slides. What looks different? Does our polytropic approximation work well? If not, where does it fail?

## c - "Stiff" vs "Soft" Equations of State

1. Plot the results of `a` and `b` together on the same plot. (You can copy/paste from the blocks above.)


In [ ]:
# hint: plot(X,Y,label='name_here') will allow you to label the individual lines and generate a legend for your plot

# plot from a (minus labels):


#plot from b (minus labels):


#labels:
plt.xlabel('Radius (km)')
plt.ylabel(r'Mass ($M_{\odot}$)')
plt.title('M vs R')

#add legend
plt.legend()

With your group, discuss:

>2. A "soft" equation of state refers to material that is more "squishy"/compressible in nature, while a "stiff" one would be less "squishy"/compressible. Which of the EoS plotted do you think is the "softer" or "stiffer" one and why? (Hint: think about how softness might affect HSE and the maximum mass)
>3. If determining the NS EoS only requires knowledge of M vs R, why haven't we figured it out yet?
>4. What observational techniques might we use to measure the masses radii of neutron stars?